<a href="https://colab.research.google.com/github/swapnilmn/Assignment_1_CS6910/blob/master/Question_2ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2. Implement a feedforward neural network which takes images from the fashion-mnist data as input and outputs a probability distribution over the 10 classes.

Your code should be flexible such that it is easy to change the number of hidden layers and the number of neurons in each hidden layer.

# Libraries 

In [1]:
from keras.datasets import fashion_mnist, mnist
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
import pandas as pd

In [2]:
class OneHotEncoder_from_scratch:
    
    def __init__(self):
        self.categories = None
    def fit(self, X):
        self.categories =[]
        for i in range(X.shape[1]):
            feature_categories =list(set(X[:, i]))
            self.categories.append(feature_categories)
            
    def transform(self, X):
        one_hot_vector = []

        for i in range(X.shape[0]):
            one_hot_row = []
            for j in range(X.shape[1]):

                category_index = self.categories[j].index(X[i, j])
                category_one_hot =[0] *len(self.categories[j])
                category_one_hot[category_index] = 1

                one_hot_row.extend(category_one_hot)
            one_hot_vector.append(one_hot_row)
        return np.array(one_hot_vector)

X = np.array([[1],[2]]) 
enc = OneHotEncoder_from_scratch()
enc.fit(X)
enc.transform(X)

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
enc.fit(train_labels.reshape(-1, 1))
enc.transform(train_labels.reshape(-1, 1))
#train_labels.reshape(-1, 1)

4422102/4422102 [==============================] - 0s 0us/step


array([[0, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [3]:
def dataset_type(dataset = 'fashion_mnist'):
  if dataset == 'fashion_mnist':
      (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
  elif dataset == 'mnist':
      (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
  else:
      raise ValueError('Invalid dataset name')
  X_train, X_val, Y_train, Y_val = train_test_split(train_images, train_labels, test_size=0.1, random_state=42)
  # Flatten input images
  train_input = X_train.reshape((X_train.shape[0], -1))
  test_input = test_images.reshape((test_images.shape[0], -1))
  val_input = X_val.reshape((X_val.shape[0], -1))

  Y_train = np.array(Y_train)
  Y_val = np.array(Y_val)
  Y_test = np.array(test_labels)

  X_train = np.array(train_input) / 255.0
  X_test = np.array(test_input) / 255.0
  X_val = np.array(val_input) / 255.0

  enc = OneHotEncoder_from_scratch()
  enc.fit(Y_train.reshape(-1, 1))
  Y_train = enc.transform(Y_train.reshape(-1, 1))
  Y_val = enc.transform(Y_val.reshape(-1, 1))
  Y_test = enc.transform(Y_test.reshape(-1, 1))

  return X_train, X_val, X_test, Y_train, Y_val, Y_test

X_train, X_val, X_test, Y_train, Y_val, Y_test = dataset_type(dataset = 'fashion_mnist')
# print(Y_train.shape, Y_val.shape, Y_test.shape)
# print(X_train.shape, X_val.shape, X_test.shape)

print(Y_train.shape, Y_val.shape, Y_test.shape)
print(X_train.shape, X_val.shape, X_test.shape)

(54000, 10) (6000, 10) (10000, 10)
(54000, 784) (6000, 784) (10000, 784)


In [4]:
# we will set a configuartion dictionary for future use for wandb seerp
# we will take input from that configuration dict

config = {'hidden_layers_size' : [32, 64, 128], # We can modify numbers of neurons each layers, parallaly it will modify number of layers
          'pre_activation_function': 'sigmoid',
          'weight_initialization_method': 'xavier',
          }
# Now we will dynamically add no of hidden layers key 
config['hidden_layers_no'] = len(config['hidden_layers_size'])


In [5]:
class FFNN:
  def __init__(self, X, Y,
               epochs=100, 
               hidden_layer_count=4,
               hidden_layers=[32, 64, 128, 256],
               learning_rate=0.001,
               batch_size=32,
               activation='tanh',
               weight_init='random',
               loss='mean_squared_error',
               weight_decay=0):
    
    self.params = {
      'inputs': X.shape[1],
      'outputs': Y.shape[1],
      'epochs': epochs,
      'hidden_layers': hidden_layer_count,
      'network_size': [X.shape[1]] + hidden_layers +[Y.shape[1]],
      'learning_rate': learning_rate,
      'batch_size': batch_size,
      'weights': {},
      'weight_init': weight_init,
      'activation_function':activation,
      'loss_function': loss,
      'lambd': weight_decay
    }

    self.update(self.params)
    np.random.seed(0)

  def update(self, params):
    for key, value in params.items():
      setattr(self, key, value)

    self.grad_derivatice={}
    self.u_w = {f'{key}{i}': 0 for i in range(1, self.hidden_layers+1) for key in ['vw','vb', 'mb', 'mw']}
    self.p_u_w = {f'{key}{i}': 0 for i in range(1, self.hidden_layers+1) for key in ['vw', 'vb', 'mb', 'mw']}

    # for creating initial weights
    if self.weight_init == 'random':
      for i in range(1, self.hidden_layers + 2):
          weight_shape = (self.network_size[i - 1], self.network_size[i])
          weight_scale = 0.1
          np.random.seed(0) # including random seet fpr reproducibilty
          self.weights[f'W{i}'] = np.random.normal(scale=weight_scale, size=weight_shape)*0.1
          # we are mulliplying ny 0.1 to get less wrights
          bias_shape = (1, self.network_size[i])
          self.weights[f'B{i}'] = np.zeros(bias_shape)

    if self.weight_init == 'Xavier':
      for i in range(1, self.hidden_layers + 2):
          weight_shape = (self.network_size[i - 1],self.network_size[i])
          weight_scale = 0.1
          np.random.seed(0)
          self.weights[f'W{i}'] = np.random.normal(scale=weight_scale,size=weight_shape)*np.sqrt(1/self.network_size[i-1])
          # we are mulliplying ny 0.1 to get less wrights
          bias_shape = (1, self.network_size[i])
          self.weights[f'B{i}'] = np.zeros(bias_shape)



  def forward_activation(self, X):
      # Define a dictionary of activation functions and their corresponding lambda functions
    activation_functions = {
        'sigmoid': lambda x: 1.0 / (1.0 + np.exp(-x)), # sigmoid activation function
        'tanh': np.tanh, # hyperbolic tangent activation function
        'ReLU': lambda x: np.maximum(0, x) # rectified linear unit (ReLU) activation function
    }
    # Get the activation function based on the value of `self.activation_function`
    activation_function = activation_functions.get(self.activation_function)
    # If the activation function is found, apply it to the input matrix `X`
    if activation_function:
        return activation_function(X)
    # If the activation function is not found, raise a ValueError indicating that it is unknown
    else:
        raise ValueError(f"Unknown activation function '{self.activation_function}'")



  def grad_activation(self, X):
    # Define a dictionary of activation function derivatives and their corresponding lambda functions
    activation_gradients = {
        'sigmoid': lambda x: x * (1 - x), # derivative of the sigmoid activation function
        'tanh': lambda x: 1 - np.square(x), # derivative of the hyperbolic tangent activation function
        'ReLU': lambda x: 1.0 * (x > 0) # derivative of the rectified linear unit (ReLU) activation function
    }
    # Get the derivative of the activation function based on the value of `self.activation_function`
    gradient_function = activation_gradients.get(self.activation_function)
    # If the derivative function is found, apply it to the input matrix `X`
    if gradient_function:
        return gradient_function(X)
    # If theerivative function is not found, raise a alueError indicating that it is unknown
    else:
        raise ValueError(f"Unknown activation function '{self.activation_function}'")


  def softmax(self, X):
    exps =np.exp(X - np.max(X, axis=1, keepdims=True)) # to reduce long numbers
    return  exps /np.sum(exps, axis=1, keepdims=True)


  def forward_pass(self, X, weights=None):
    # X: shape (batch_size, input_dim)
    if weights is None:
        weights = self.weights

    # Initialize dictionaries to store intermediate outputs
    self.z = {}
    self.h = {}
    self.h[0] = X

    # Perform forward pass through hidden layers
    for i in range(self.hidden_layers):
        # Compute weighted sum of inputs and biases
        z_i = self.h[i] @ weights[f'W{i+1}']
        z_i = z_i + weights[f'B{i+1}']
        self.z[i+1] = z_i
        # Apply activation function
        h_i = self.forward_activation(z_i)
        self.h[i+1] = h_i

    # Compute final output
    z_final = self.h[self.hidden_layers] @ weights[f'W{self.hidden_layers+1}']
    z_final = z_final + weights[f'B{self.hidden_layers+1}']
    self.z[self.hidden_layers+1] = z_final

    # Apply softmax activation functionto final output
    h_final = self.softmax(z_final)
    self.h[self.hidden_layers+1] = h_final
    # Return final output
    return h_final

  
model = FFNN(X_train, Y_train,
                  epochs = 1, 
                  hidden_layer_count = 4,
                  hidden_layers =  [32, 64, 128, 256],
                  learning_rate = 0.001,
                  batch_size = 32,
                  activation='ReLU',
                  weight_init='random',
                  loss = 'cross_entropy')
model.forward_pass(X_train)[1]

array([0.09999944, 0.09999937, 0.10000029, 0.10000046, 0.09999999,
       0.1000001 , 0.09999989, 0.10000012, 0.09999968, 0.10000066])